In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np

#gpu_options = tf.gpu_options(per_process_gpu_memory_fraction=0.3333)
#sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [7]:
DATA_PATH='/home/yifei/Downloads/PetImages'
CATEGORIES=['Dog','Cat']
IMG_SIZE=100

In [31]:
x_train=[] #create an empty list for training data
def create_training_data():
    for category in CATEGORIES:
        my_path=os.path.join(DATA_PATH,category)
        classnum=CATEGORIES.index(category) #gives you the index at that category (0 or 1)
        
        for image in os.listdir(my_path):
            try:
                img_array=cv2.imread(os.path.join(my_path,image),cv2.IMREAD_GRAYSCALE) #this will be in grayscale
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE)) #resize every image before passing it in
                x_train.append([new_array,classnum])
            except Exception as e:
                pass
            

create_training_data()
print(len(x_train))

24946


In [55]:
#shuffle data 
import random
random.shuffle(x_train)

print(x_train[0])

[array([[152, 154, 154, ..., 245, 245, 246],
       [152, 154, 154, ..., 247, 247, 248],
       [153, 153, 154, ..., 246, 247, 248],
       ...,
       [155, 209, 185, ..., 218, 223, 229],
       [162, 157, 191, ..., 224, 198, 206],
       [187, 156, 151, ..., 210, 222, 220]], dtype=uint8), 1]


In [84]:
import pickle
x=[]
y=[]
for features, label in x_train:
    x.append(features) 
    y.append(label) #create lists of features and labels 
#we cannot pass a list into neural network, we need to use a numpy array
x=np.array(x).reshape(-1,IMG_SIZE,IMG_SIZE,1) #1 is for grayscale, -1 represents unknown dimension for np to resolve
print(x.shape)

#save data
pickle_out=open('x.pickle','wb')
pickle.dump(x,pickle_out)
pickle_out.close();

pickle_out=open('y.pickle','wb')
pickle.dump(y,pickle_out)
pickle_out.close();

print(len(y))

(24946, 100, 100, 1)
24946


In [5]:
import pickle
pickle_in=open('x.pickle','rb')
x=pickle.load(pickle_in)

pickle_in=open('y.pickle','rb')
y=pickle.load(pickle_in)

x=x/255.0

In [6]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.95):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True
        
def createModel():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=x.shape[1:]),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(), #flattens the dataset for our input layer
        tf.keras.layers.Dense(512,activation=tf.nn.relu), #hidden layer
        tf.keras.layers.Dense(1,activation='sigmoid') #sigmoid for final layer
    ])
    return model

call=myCallback()
model=createModel()


Instructions for updating:
Colocations handled automatically by placer.


In [8]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model.fit(x, y, batch_size=180, validation_split=0.1, epochs=10, callbacks=[call])

Train on 22451 samples, validate on 2495 samples
Epoch 1/10
22451/22451 [==============================] - 863s 38ms/sample - loss: 0.6153 - acc: 0.6554 - val_loss: 0.5454 - val_acc: 0.7423
Epoch 2/10
22451/22451 [==============================] - 888s 40ms/sample - loss: 0.4981 - acc: 0.7589 - val_loss: 0.4861 - val_acc: 0.7659
Epoch 3/10
22451/22451 [==============================] - 730s 33ms/sample - loss: 0.4481 - acc: 0.7883 - val_loss: 0.4968 - val_acc: 0.7647
Epoch 4/10
22451/22451 [==============================] - 686s 31ms/sample - loss: 0.4067 - acc: 0.8146 - val_loss: 0.4772 - val_acc: 0.7832
Epoch 5/10
22451/22451 [==============================] - 749s 33ms/sample - loss: 0.3604 - acc: 0.8397 - val_loss: 0.4732 - val_acc: 0.7924
Epoch 6/10
22451/22451 [==============================] - 531s 24ms/sample - loss: 0.3245 - acc: 0.8588 - val_loss: 0.4862 - val_acc: 0.7812
Epoch 7/10
22451/22451 [==============================] - 501s 22ms/sample - loss: 0.2645 - acc: 0.8896 -

In [9]:
model.save('cats and dogs')

In [2]:
new_model=tf.keras.models.load_model('cats and dogs')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [5]:
CATEGORIES=['Dog','Cat']
#takes the filepath of an image and resizes it to become an array
def prepare(filepath):
    IMG_SIZE=100
    img_array=cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
    new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE)) #resize it so it's 100x100
    return new_array.reshape(-1,IMG_SIZE,IMG_SIZE,1) #reshape because tf needs this format

In [6]:
dog_array=prepare('dog2.jpeg')
prediction=new_model.predict(dog_array)
print(CATEGORIES[int(prediction[0][0])])

(100, 100)
Dog
